<a href="https://colab.research.google.com/github/WangKun00293/bart-bert-project/blob/main/BERT_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
from datasets import load_dataset
ag_news = load_dataset("ag_news")
## load dataset
import pandas as pd
df_train=ag_news["train"].to_pandas()
test=ag_news["test"].to_pandas()
from sklearn.model_selection import train_test_split
sample = df_train.sample(n=10000, random_state=1)
train, val = train_test_split(sample, test_size=0.1, random_state=311)
train_0=train[train["label"]==0].sample(n=500, random_state=323)
train_1=train[train["label"]==1].sample(n=800, random_state=323)
train_2=train[train["label"]==2].sample(n=1200, random_state=323)
train_3=train[train["label"]==3].sample(n=1500, random_state=323)
df_train=pd.concat([train_0,train_1,train_2,train_3]).sample(frac=1, random_state=324).reset_index(drop=True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.4 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Data preparation

In [ ]:
import re
def clean_text(text):
    text = re.sub(r'http\S+|www.\S+', '', text)
    text = re.sub(r'[&\\@#\$%\^*()_+={}\[\]:;"\'<>\|`~]', '', text)

    return text

In [ ]:
df_train['text'] = df_train['text'].apply(clean_text)
test['text'] = test['text'].apply(clean_text)

In [ ]:
# don't need prompt for BERT, add it for fine-tuning BART
label_map = {0: "Write a World News", 1: "Write a Sport News", 2: "Write a Business News", 3: "Write a Science/Technology News"}
df_train['prompt'] = df_train['label'].map(label_map)
df_remain, df_BART= train_test_split(df_train, test_size=0.5, random_state=325)
df_test, df_val = train_test_split(test,test_size=0.5,random_state=325)
df_remain=df_remain.reset_index(drop=True)
df_BART=df_BART.reset_index(drop=True)
df_test=df_test.reset_index(drop=True)
df_val=df_val.reset_index(drop=True)

## Base-line model with BERT

In [ ]:
from transformers import BertTokenizer
from transformers import BertForSequenceClassification
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=4)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
class TextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.targets = dataframe.label
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors="pt",
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'labels': torch.tensor(self.targets[index], dtype=torch.long)
        }


In [ ]:
MAX_LENGTH = 320
BATCH_SIZE = 16
train_dataset = TextDataset(dataframe=df_train, tokenizer=bert_tokenizer, max_len=MAX_LENGTH)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False)
val_dataset = TextDataset(dataframe=df_val, tokenizer=bert_tokenizer, max_len=MAX_LENGTH)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_dataset = TextDataset(dataframe=df_test, tokenizer=bert_tokenizer, max_len=MAX_LENGTH)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
import torch
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_model.to(device)
learning_rate = 2e-05
optimizer = torch.optim.AdamW(params=bert_model.parameters(), lr=learning_rate)
epochs = 5
bert_model.train()
import os

# Directory where you want to save your models
model_save_path = "/content/drive/MyDrive/planB/bert"
os.makedirs(model_save_path, exist_ok=True)

for epoch in range(epochs):
    bert_model.train()
    total_loss = 0
    # Training loop
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = bert_model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_train_loss = total_loss / len(train_loader)

    # Validation loop
    bert_model.eval()  # Switch to evaluation mode
    total_val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = bert_model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_val_loss += loss.item()
    avg_val_loss = total_val_loss / len(val_loader)

    print(f"Epoch: {epoch+1}, Training Loss: {avg_train_loss}, Validation Loss: {avg_val_loss}")

    # Save model after each epoch
    epoch_save_path = os.path.join(model_save_path, f'BERT402_epoch_{epoch+1}')
    os.makedirs(epoch_save_path, exist_ok=True)
    bert_model.save_pretrained(epoch_save_path)
    bert_tokenizer.save_pretrained(epoch_save_path)


Epoch: 1, Training Loss: 0.4864871038645506, Validation Loss: 0.35436977159638866
Epoch: 2, Training Loss: 0.2045171178430319, Validation Loss: 0.325832781248859
Epoch: 3, Training Loss: 0.11916310729086399, Validation Loss: 0.35660128549774406
Epoch: 4, Training Loss: 0.06591740463860334, Validation Loss: 0.411454915129697
Epoch: 5, Training Loss: 0.03962298171687871, Validation Loss: 0.4795967113146433


# load model and evaluate

In [ ]:
from transformers import BertTokenizer
from transformers import BertForSequenceClassification
bert_tokenizer = BertTokenizer.from_pretrained('/content/drive/MyDrive/planB/bert/BERT402_epoch_2')
bert_model = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/planB/bert/BERT402_epoch_2', num_labels=4)

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
class TextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.targets = dataframe.label
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors="pt",
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'labels': torch.tensor(self.targets[index], dtype=torch.long)
        }


In [ ]:
MAX_LENGTH = 320
BATCH_SIZE = 16
test_dataset = TextDataset(dataframe=df_test, tokenizer=bert_tokenizer, max_len=MAX_LENGTH)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
val_dataset = TextDataset(dataframe=df_val, tokenizer=bert_tokenizer, max_len=MAX_LENGTH)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np

def evaluate(model, dataloader):
    model.eval()  # Put the model in evaluation mode

    predictions, true_labels = [], []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)

            logits = outputs.logits.detach().cpu().numpy()
            label_ids = labels.to('cpu').numpy()

            predictions.extend(np.argmax(logits, axis=1).flatten())
            true_labels.extend(label_ids.flatten())

    avg_accuracy = accuracy_score(true_labels, predictions)
    print(f'Validation Accuracy: {avg_accuracy}')

    # Detailed classification report
    print("\nClassification Report:\n", classification_report(true_labels, predictions, target_names=['Class1', 'Class2', 'Class3', 'Class4']))

    # Confusion Matrix
    print("Confusion Matrix:\n", confusion_matrix(true_labels, predictions))

In [ ]:
import torch
import os
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_model.to(device)
evaluate(bert_model, val_loader)

Validation Accuracy: 0.8981578947368422

Classification Report:
               precision    recall  f1-score   support

      Class1       0.97      0.83      0.89       965
      Class2       0.96      0.98      0.97       972
      Class3       0.84      0.87      0.85       933
      Class4       0.84      0.92      0.88       930

    accuracy                           0.90      3800
   macro avg       0.90      0.90      0.90      3800
weighted avg       0.90      0.90      0.90      3800

Confusion Matrix:
 [[800  31  79  55]
 [  6 949  11   6]
 [ 13   4 809 107]
 [ 10   4  61 855]]


In [ ]:
evaluate(bert_model, test_loader)

Validation Accuracy: 0.9078947368421053

Classification Report:
               precision    recall  f1-score   support

      Class1       0.96      0.87      0.91       935
      Class2       0.97      0.98      0.98       928
      Class3       0.86      0.86      0.86       967
      Class4       0.85      0.92      0.88       970

    accuracy                           0.91      3800
   macro avg       0.91      0.91      0.91      3800
weighted avg       0.91      0.91      0.91      3800

Confusion Matrix:
 [[814  25  56  40]
 [  4 911   8   5]
 [ 22   0 832 113]
 [  8   2  67 893]]


## use test set

In [ ]:
from transformers import BertTokenizer
from transformers import BertForSequenceClassification
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=4)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
class TextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.targets = dataframe.label
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors="pt",
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'labels': torch.tensor(self.targets[index], dtype=torch.long)
        }


In [ ]:
MAX_LENGTH = 320
BATCH_SIZE = 16
train_dataset = TextDataset(dataframe=test, tokenizer=bert_tokenizer, max_len=MAX_LENGTH)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False)
val_dataset = TextDataset(dataframe=df_train, tokenizer=bert_tokenizer, max_len=MAX_LENGTH)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)


In [ ]:
import torch
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_model.to(device)
learning_rate = 2e-05
optimizer = torch.optim.AdamW(params=bert_model.parameters(), lr=learning_rate)
epochs = 5
bert_model.train()
import os

# Directory where you want to save your models
model_save_path = "/content/drive/MyDrive/planB/bert"
os.makedirs(model_save_path, exist_ok=True)

for epoch in range(epochs):
    bert_model.train()
    total_loss = 0
    # Training loop
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = bert_model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_train_loss = total_loss / len(train_loader)

    # Validation loop
    bert_model.eval()  # Switch to evaluation mode
    total_val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = bert_model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_val_loss += loss.item()
    avg_val_loss = total_val_loss / len(val_loader)

    print(f"Epoch: {epoch+1}, Training Loss: {avg_train_loss}, Validation Loss: {avg_val_loss}")

    # Save model after each epoch
    epoch_save_path = os.path.join(model_save_path, f'BERT524_epoch_{epoch+1}')
    os.makedirs(epoch_save_path, exist_ok=True)
    bert_model.save_pretrained(epoch_save_path)
    bert_tokenizer.save_pretrained(epoch_save_path)


Epoch: 1, Training Loss: 0.42566792384966423, Validation Loss: 0.27410541274398564
Epoch: 2, Training Loss: 0.19565956631185194, Validation Loss: 0.30037309251725675


KeyboardInterrupt: 

In [ ]:
from transformers import BertTokenizer
from transformers import BertForSequenceClassification
bert_tokenizer = BertTokenizer.from_pretrained('/content/drive/MyDrive/planB/bert/BERT524_epoch_1')
bert_model = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/planB/bert/BERT524_epoch_1', num_labels=4)
val_dataset = TextDataset(dataframe=df_train, tokenizer=bert_tokenizer, max_len=MAX_LENGTH)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
import torch
import os
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_model.to(device)
evaluate(bert_model, val_loader)

Validation Accuracy: 0.9065

Classification Report:
               precision    recall  f1-score   support

      Class1       0.87      0.87      0.87       500
      Class2       0.96      0.99      0.97       800
      Class3       0.88      0.88      0.88      1200
      Class4       0.91      0.90      0.90      1500

    accuracy                           0.91      4000
   macro avg       0.91      0.91      0.91      4000
weighted avg       0.91      0.91      0.91      4000

Confusion Matrix:
 [[ 436   18   37    9]
 [   2  791    5    2]
 [  17   10 1053  120]
 [  45    7  102 1346]]
